Przetwarzanie Danych
===========

W celu przetwarzania danych wykorzystałem kod z kernela helpers udostępnionego na Kagglu.
Jedyną zmianą jakiej dokonałem było dodanie poniższej linijki w celu przejścia do skali logarytmicznej na melspectrogramie (decybele). Pozwoliło to uzyskać wyniki średnio wyniki lepsze o 3-4%.



In [ ]:
log_spec = librosa.core.power_to_db(spectrogram, ref=np.median)

Architektura
=========

Moje rozwiązanie problemu detekcji ptaków na nagraniach, wykorzystuje konwolucyjne sieci neuronowe. 
Konkretniej sieć składającą się z 3 warstw konwolucyjnych oraz dwóch warstw liniowych. 
Warswy konwolucyjne miały kolejno 16, 32 oraz 32 filtry, a także kernele rozmiarów 3x3, 3x3 oraz 2x2. Reszta szczegołów w notatniku FinalModel.

Inne metody
===========

W rozwiązaniu wykorzystałem ensambling. Finalnie użyłem 10 modeli (ta sama architektura, ale inne wagi), które uzyskiwały najlepsze wyniki na zbiorze walidacyjnym. A następnie uśredniłem ich predykcje co pozwoliło uzyskać na kagglu wynik na poziomie ~0.94 (Pojedyncze modele miały wyniki między 0.91 a 0.93 na zbiorze walidacyjnym).

Generator predykcji
===========

In [7]:
import os
os.chdir('.')
from models import *

In [ ]:
paths = []
for i in range(1,11):
    paths.append('saved_model/saved{}.pkl'.format(i))

n_models = 10
pretrained = [True] * n_models
models = [CNN_Wrapper(i+1, pretrained[i], paths[i]) for i in range(n_models)]

In [14]:
X_test_d = np.load('test_data.pkl.npy')

vot_clf = VotingClassifier(models)

In [ ]:
vot_clf.fit(None, None)

In [ ]:
labels = vot_clf.predict(X_test_d)

In [ ]:
final_labels = vot_clf.voting(labels)